<head>
    <h1>Semana 9 - LAB 3<h1>
<head>
    <div>
    <img align="left" alt="Geron-Mattoso" height="100" width="150" 
src="https://spark.apache.org/images/spark-logo-rev.svg">
    </div>

In [27]:
import os
caminho = os.getcwd()
print(caminho)

/home/geron/Compass


In [28]:
from pyspark import SparkContext, SQLContext
from pyspark.sql import *
from pyspark.sql.functions import *

import os
caminho = os.getcwd()
print(caminho)


spark = SparkSession.builder.appName("Exercicios").getOrCreate()

sqlContext = SQLContext(spark.sparkContext)



/home/geron/Compass


In [29]:
df_nomes = spark.read.csv(caminho + '/nomes.csv', header = True, inferSchema=True)

In [30]:
type(df_nomes)

pyspark.sql.dataframe.DataFrame

In [31]:
df_nomes.show(5)

+--------+----+-----+----+
|    nome|sexo|total| ano|
+--------+----+-----+----+
|Jennifer|   F|54336|1983|
| Jessica|   F|45278|1983|
|  Amanda|   F|33752|1983|
|  Ashley|   F|33292|1983|
|   Sarah|   F|27228|1983|
+--------+----+-----+----+
only showing top 5 rows



In [32]:
df_nomes.printSchema()

root
 |-- nome: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- total: integer (nullable = true)
 |-- ano: integer (nullable = true)



In [33]:
df_nomes.select(df_nomes.nome, df_nomes.sexo).show(5)

+--------+----+
|    nome|sexo|
+--------+----+
|Jennifer|   F|
| Jessica|   F|
|  Amanda|   F|
|  Ashley|   F|
|   Sarah|   F|
+--------+----+
only showing top 5 rows



In [34]:
df_select = df_nomes.select(df_nomes.nome,
                            df_nomes.ano,
                            df_nomes.ano > 1950,
                            df_nomes.ano + 1000)
df_select.show(5)

+--------+----+------------+------------+
|    nome| ano|(ano > 1950)|(ano + 1000)|
+--------+----+------------+------------+
|Jennifer|1983|        true|        2983|
| Jessica|1983|        true|        2983|
|  Amanda|1983|        true|        2983|
|  Ashley|1983|        true|        2983|
|   Sarah|1983|        true|        2983|
+--------+----+------------+------------+
only showing top 5 rows



In [35]:
df_nomes.registerTempTable("pessoas")

In [36]:
sqlContext.sql("select count (*) from pessoas").show()

+--------+
|count(1)|
+--------+
| 1825433|
+--------+



In [37]:
sqlContext.sql("select nome, ano, ano+1000 as futuro FROM pessoas").show(5)

+--------+----+------+
|    nome| ano|futuro|
+--------+----+------+
|Jennifer|1983|  2983|
| Jessica|1983|  2983|
|  Amanda|1983|  2983|
|  Ashley|1983|  2983|
|   Sarah|1983|  2983|
+--------+----+------+
only showing top 5 rows



In [38]:
df_nomes.filter(df_nomes.ano > 1990).select(df_nomes.nome, df_nomes.ano).show(5)

+--------+----+
|    nome| ano|
+--------+----+
|  Ashley|1991|
| Jessica|1991|
|Brittany|1991|
|  Amanda|1991|
|Samantha|1991|
+--------+----+
only showing top 5 rows



In [39]:
df_nomes.filter(df_nomes.ano > 1990)\
    .select(df_nomes.nome, df_nomes.ano)\
        .orderBy(df_nomes.ano, df_nomes.nome).show(5)

df_nomes.filter(df_nomes.ano > 1990)\
    .select(df_nomes.nome, df_nomes.ano)\
        .orderBy(df_nomes.ano.desc(), df_nomes.nome).show(5)



+-------+----+
|   nome| ano|
+-------+----+
| Aaisha|1991|
| Aakash|1991|
|Aaliyah|1991|
| Aamber|1991|
|Aaminah|1991|
+-------+----+
only showing top 5 rows

+---------+----+
|     nome| ano|
+---------+----+
|    Aaban|2014|
|    Aabha|2014|
|Aabriella|2014|
|    Aadam|2014|
|    Aadan|2014|
+---------+----+
only showing top 5 rows



In [40]:
df_nomes.select("nome", "ano", (df_nomes.ano > 2000).alias('recente')).show(5)

+--------+----+-------+
|    nome| ano|recente|
+--------+----+-------+
|Jennifer|1983|  false|
| Jessica|1983|  false|
|  Amanda|1983|  false|
|  Ashley|1983|  false|
|   Sarah|1983|  false|
+--------+----+-------+
only showing top 5 rows



In [41]:
spark.sql("SELECT nome, ano, ano > 2000 as recente FROM pessoas").show(5)

+--------+----+-------+
|    nome| ano|recente|
+--------+----+-------+
|Jennifer|1983|  false|
| Jessica|1983|  false|
|  Amanda|1983|  false|
|  Ashley|1983|  false|
|   Sarah|1983|  false|
+--------+----+-------+
only showing top 5 rows



In [42]:
df_nomes.groupBy('ano').count().show(5)

+----+-----+
| ano|count|
+----+-----+
|1990|24713|
|2003|31173|
|1988|22358|
|1997|26965|
|1994|25997|
+----+-----+
only showing top 5 rows



In [43]:
df_nomes.groupBy('sexo').count().show(5)

+----+-------+
|sexo|  count|
+----+-------+
|   F|1081683|
|   M| 743750|
+----+-------+



#### EXERCÍCIO 1: Faça o agrupamento sexo e realize a contagem. Responda as seguintes perguntas:

•	Qual o sexo que mais aparece?

•	Qual foi a diferença entre o sexo Masculino (F) e do Feminino (F)?


In [44]:
'''Qual sexo mais aparece'''
df_quant = spark.sql("select sexo, count(sexo) as count from pessoas group by sexo")

df_quant.show(1)

'''qual a diferença entrer sexos'''

df_quant.createOrReplaceTempView("quant")

spark.sql("SELECT MAX(count)-MIN(count) AS diferenca FROM quant").show()


+----+-------+
|sexo|  count|
+----+-------+
|   F|1081683|
+----+-------+
only showing top 1 row



+---------+
|diferenca|
+---------+
|   337933|
+---------+



##### EXERCÍCIO 2:  Crie uma nova coluna no DataFrame que contenha apenas números. Nesta coluna chamada RANKING deverá ter o número que representa a colocação do nome na lista, sendo o número 1 o nome mais usado em todos os anos.

In [45]:
''''
agrupar nomes por quantidade
Criar coluna com id e ordenar
'''
df_ranking = df_nomes.groupBy("nome")\
    .sum("total").orderBy(col("sum(total)").desc())\
        .withColumn("Ranking", monotonically_increasing_id()+1).show()


+-----------+----------+-------+
|       nome|sum(total)|Ranking|
+-----------+----------+-------+
|      James|   5129096|      1|
|       John|   5106590|      2|
|     Robert|   4816785|      3|
|    Michael|   4330805|      4|
|       Mary|   4130441|      5|
|    William|   4071368|      6|
|      David|   3590557|      7|
|     Joseph|   2580687|      8|
|    Richard|   2564867|      9|
|    Charles|   2376700|     10|
|     Thomas|   2291517|     11|
|Christopher|   2004177|     12|
|     Daniel|   1876880|     13|
|  Elizabeth|   1606282|     14|
|   Patricia|   1575529|     15|
|    Matthew|   1558671|     16|
|   Jennifer|   1467573|     17|
|     George|   1464430|     18|
|      Linda|   1454599|     19|
|    Barbara|   1437083|     20|
+-----------+----------+-------+
only showing top 20 rows



EXERCÍCIO 3:  Crie 03 novas colunas no DataFrame que contenha apenas Sim e Não. Cada coluna deve-se chamar DECADA_80, DECADA_90, DECADA_00 e o conteúdo deve ser SIM ou NÃO  de acordo com o ano definido na tabela.

In [46]:
#https://sparkbyexamples.com/pyspark/pyspark-when-otherwise/

df_decadaSN = df_nomes\
    .withColumn('DECADA_80', when((df_nomes.ano >= 1980), 'Sim').when ((df_nomes.ano < 1990), 'Nao').otherwise("df_nomes.ano"))\
        .withColumn('DECADA_90', when((df_nomes.ano >= 1990), 'Sim').when ((df_nomes.ano < 2000), 'Nao').otherwise("df_nomes.ano"))\
            .withColumn('DECADA_00', when((df_nomes.ano >= 2000), 'Sim').when((df_nomes.ano < 2010), 'Nao').otherwise("df_nomes.ano"))\

df_decadaSN.show(5)

+--------+----+-----+----+---------+---------+---------+
|    nome|sexo|total| ano|DECADA_80|DECADA_90|DECADA_00|
+--------+----+-----+----+---------+---------+---------+
|Jennifer|   F|54336|1983|      Sim|      Nao|      Nao|
| Jessica|   F|45278|1983|      Sim|      Nao|      Nao|
|  Amanda|   F|33752|1983|      Sim|      Nao|      Nao|
|  Ashley|   F|33292|1983|      Sim|      Nao|      Nao|
|   Sarah|   F|27228|1983|      Sim|      Nao|      Nao|
+--------+----+-----+----+---------+---------+---------+
only showing top 5 rows



EXERCÍCIO 4:  Remova do Dataframe a coluna DECADA_00 e mostrar o novo Dataframe.

In [47]:
df_decadaremoved00 = df_decadaSN.drop('DECADA_00')

df_decadaremoved00.show(5)

+--------+----+-----+----+---------+---------+
|    nome|sexo|total| ano|DECADA_80|DECADA_90|
+--------+----+-----+----+---------+---------+
|Jennifer|   F|54336|1983|      Sim|      Nao|
| Jessica|   F|45278|1983|      Sim|      Nao|
|  Amanda|   F|33752|1983|      Sim|      Nao|
|  Ashley|   F|33292|1983|      Sim|      Nao|
|   Sarah|   F|27228|1983|      Sim|      Nao|
+--------+----+-----+----+---------+---------+
only showing top 5 rows



EXERCÍCIO 5: Gerar dois arquivos CSV. O primeiro com os nomes do sexo masculino nascidos na década de 80. O segundo com os nomes do sexo feminino nascidos na década de 80.

In [48]:

df_decadaremoved00.filter((df_decadaremoved00.DECADA_80 == 'Sim') & (df_decadaremoved00.sexo == 'M'))\
    .drop('DECADA_80','DECADA_90','ano', 'total', 'sexo')\
    .write.csv('./Masculino_DECADA_80', mode="overwrite")

df_decadaremoved00.filter((df_decadaremoved00.DECADA_80 == 'Sim') & (df_decadaremoved00.sexo == 'F'))\
    .drop('DECADA_80','DECADA_90','ano', 'total', 'sexo')\
    .write.csv('./Feminino_DECADA_80', mode="overwrite")